In [5]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

from scipy.spatial import cKDTree
import haversine as hs  # pip install haversine



# Nearest distance with haversine function

In [4]:
# import hospital data as geodataframe
gdf_spital = gpd.read_file(r'C:\Users\gigim\Documents\GitHub\mlp\geodata\\data_spital.gpkg')
gdf_laerm = gpd.read_file(r'C:\Users\gigim\Documents\GitHub\mlp\geodata\\data_strassenlaerm.gpkg')
gdf_ov = gpd.read_file(r'C:\Users\gigim\Documents\GitHub\mlp\geodata\\Haltestellen_des_offentlichen_Verkehrs_-OGD.gpkg')

# read csv file of addresses with long and lat
df_24 = pd.read_csv('immoscout24_geocoded.csv', delimiter='|')



In [12]:
gdf_ov = gpd.read_file(r'C:\Users\gigim\Documents\GitHub\mlp\geodata\\Haltestellen_des_offentlichen_Verkehrs_-OGD.gpkg')

In [7]:

# remove nan values in geodaraframe
print(df_24.shape)
# Filter out rows where 'geometry' is NaN
df_24.dropna(subset=['longitude'], inplace=True)
print(df_24.shape)



(641, 31)
(637, 31)


In [13]:
#convert geoDataFrame to DF
def todf(gdf):
    # Transform to WGS 84 CRS
    gdf = gdf.to_crs(epsg=4326)
    # First ensure that your geometries are points
    assert all(gdf.geometry.geom_type == 'Point'), "All geometries must be Points."

    # Create longitude and latitude columns from the geometry
    gdf['longitude'] = gdf.geometry.x
    gdf['latitude'] = gdf.geometry.y

    # Now convert your GeoDataFrame to a DataFrame
    df = pd.DataFrame(gdf.drop(columns='geometry'))

df_spital = todf(gdf_spital)
df_laerm = todf(gdf_laerm)
df_ov = todf(gdf_ov)


In [ ]:
# Calculate distance using haversine (Quelle: https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b, https://pypi.org/project/haversine/)

def distance(df_add, df_24 = df_24):

    for index, row in df_24.iterrows():
        loc1 = (row['latitude'], row['longitude'])
        print(loc1)
        min_distance = float('inf')
        for index2, row2 in df_add.iterrows():
            loc2 = (row2['latitude'], row2['longitude'])
            print(loc2)
            distance = hs.haversine(loc1,loc2)
            if distance < min_distance:
                df_24.at[index, 'distance_hosp'] = distance
                min_distance = distance
    return df_24

    

In [ ]:
# with kd tree by chat gpt

def calculate_distance(df_add, df_24):
    # Convert DataFrame columns to numpy arrays for KD-Tree
    tree = cKDTree(df_add[['latitude', 'longitude']].to_numpy())
    distances, indices = tree.query(df_24[['latitude', 'longitude']].to_numpy(), k=1)

    # Convert distances from degrees (output by KD-Tree) to kilometers using haversine
    df_24['distance_hosp'] = [haversine(df_24[['latitude', 'longitude']].iloc[i], df_add[['latitude', 'longitude']].iloc[indices[i]]) for i in range(len(indices))]

    return df_24